In [18]:
import numpy as np
import pandas as pd

from glm.families import Gaussian, Bernoulli, Poisson, Gamma
from glm.glm import GLM
from glm.simulation import Simulation

import statsmodels.api as sm
import statsmodels

In [48]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(-1, 1, size=N)
X[:, 2] = 0.0
nu = 1 - 2*X[:, 1] + X[:, 2] + np.random.normal(0.0, 1.0, size=N)

## Linear Model

In [49]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

LinAlgError: Singular matrix

In [ ]:
model.coef_

In [50]:
model.coef_covariance_matrix_

ValueError: Parameter covariances can only be estimated for afit model.

In [51]:
model.coef_standard_error_

ValueError: Parameter covariances can only be estimated for afit model.

In [52]:
model.p_values_

ValueError: Parameter covariances can only be estimated for afit model.

In [53]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     690.3
Date:                Tue, 15 May 2018   Prob (F-statistic):          4.56e-116
Time:                        11:25:06   Log-Likelihood:                -1713.3
No. Observations:                1000   AIC:                             3431.
Df Residuals:                     998   BIC:                             3440.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0273      0.043     24.169      0.0

/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1471: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1042: RuntimeWarning: invalid value encountered in absolute
  return stats.t.sf(np.abs(self.tvalues), df_resid)*2
/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/matthewdrury/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/matthewdrury/anaconda3/lib/python3

In [26]:
res.params

array([ 1.06908135, -1.953554  ,  0.96926712])

In [27]:
res.bse

array([ 0.04387157,  0.07651736,  0.07547259])

## Linear Regression With Formula

In [28]:
df = pd.DataFrame(
    np.concatenate([X[:, 1:], y.reshape(-1, 1)], axis=1),
    columns=['Moshi', 'SwimSwim', 'y'])

In [29]:
df.shape

(1000, 3)

In [30]:
model = GLM(family=Gaussian())
model.fit(df, formula='y ~ Moshi + SwimSwim')

In [31]:
model.coef_

array([ 1.06908135, -1.953554  ,  0.96926712])

In [32]:
model.summary()

Gaussian GLM Model Summary.
Name         Parameter Estimate  Standard Error
-----------------------------------------------
Intercept                  1.07            0.04
Moshi                     -1.95            0.08
SwimSwim                   0.97            0.08


## Run some simulations off the linear model.

In [33]:
s = Simulation(model)

In [34]:
s.sample(X)

array([[ 2.41340333,  0.02702069, -1.26247705, ..., -0.76463248,
        -0.28082694,  2.80473032],
       [-0.0183135 ,  2.24273984,  2.4460763 , ...,  0.53874883,
        -2.5605364 ,  1.84882278],
       [-0.49258404,  2.59417541,  0.82458661, ...,  0.03985761,
        -2.30047957, -0.44607451],
       ..., 
       [ 2.54217021,  2.0792961 ,  0.65058769, ...,  0.79786234,
        -1.74874608,  1.56058894],
       [ 1.91891425,  3.71639818, -1.37250842, ...,  1.70269483,
        -0.56837403,  2.21455692],
       [-0.05832713,  1.7854249 , -1.39736063, ..., -1.49777483,
         0.11265071,  1.97547136]])

In [35]:
models = s.parametric_bootstrap(X, n_sim=10)
for model in models:
    print(model.coef_)

[ 1.01130808 -1.94214993  0.99286685]
[ 1.08131714 -1.98856395  0.97575392]
[ 1.00830823 -1.94550728  1.00260243]
[ 1.08213834 -1.93227919  0.99824853]
[ 1.03954686 -1.85879806  0.99507033]
[ 1.06101224 -1.94422313  0.9011463 ]
[ 1.06620906 -2.00773976  0.94729269]
[ 1.09028166 -1.92107768  0.86845735]
[ 1.0692615  -1.92410722  0.91898719]
[ 0.99688073 -2.0924236   0.82665131]


In [36]:
models = s.non_parametric_bootstrap(X, y, n_sim=10)
for model in models:
    print(model.coef_)

[ 1.00369469 -1.89237278  1.00154259]
[ 1.10947362 -1.84315303  1.0648931 ]
[ 0.96535354 -1.97400379  0.94250002]
[ 1.0668595  -1.93623271  0.9202342 ]
[ 1.13216298 -1.98375475  0.85082954]
[ 1.07584556 -1.97150403  0.92416677]
[ 1.07285253 -1.7680574   0.99482327]
[ 1.02705214 -2.04499992  0.87698839]
[ 1.07229389 -1.99685788  0.80947828]
[ 1.09169521 -1.94533102  0.88409735]


## Linear Model with Sample Weights

In [37]:
sample_weights = np.random.uniform(0, 2, size=N)

In [38]:
model = GLM(family=Gaussian())
model = model.fit(X, y, sample_weights=sample_weights)

In [39]:
model.coef_

array([ 1.06868439, -1.95861928,  0.994466  ])

## Logistic Model

In [40]:
p = 1 / (1 + np.exp(-nu))
y_logistic = np.random.binomial(1, p=p, size=N)

In [41]:
model = GLM(family=Bernoulli())
model.fit(X, y_logistic)

In [42]:
model.coef_

array([ 0.87424113, -1.54587675,  1.07593918])

In [43]:
model.dispersion_

array(1.0)

In [44]:
model.coef_covariance_matrix_

array([[ 0.00618879, -0.00334314,  0.0017015 ],
       [-0.00334314,  0.02068844, -0.0035327 ],
       [ 0.0017015 , -0.0035327 ,  0.01763526]])

In [45]:
model.coef_standard_error_

array([ 0.07866883,  0.14383476,  0.13279782])

In [47]:
model.p_values_

array([  5.42703117e-29,   3.04156359e-27,   4.92171535e-16])

In [46]:
mod = sm.Logit(y_logistic, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.535719
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      997
Method:                           MLE   Df Model:                            2
Date:                Tue, 15 May 2018   Pseudo R-squ.:                  0.1562
Time:                        11:24:12   Log-Likelihood:                -535.72
converged:                       True   LL-Null:                       -634.88
                                        LLR p-value:                 8.570e-44
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8742      0.079     11.113      0.000       0.720       1.028
x1            -1.5459      0.

In [56]:
s = Simulation(model)

In [57]:
s.sample(X, n_sim=10)

array([[ 0.,  1.,  0., ...,  1.,  0.,  1.],
       [ 1.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  1.,  1., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  1.,  1., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  0.,  0.]])

In [58]:
for model in s.parametric_bootstrap(X, n_sim=10):
    print(model.coef_)

[ 0.99422083 -1.96298402  1.05235721]
[ 0.96443709 -1.99466032  1.03640602]
[ 1.00035789 -2.06225649  1.00416288]
[ 0.95647016 -1.9472292   1.09357851]
[ 0.93641543 -1.96767443  1.01963135]
[ 0.96543839 -1.95048238  1.0720573 ]
[ 0.97230804 -1.96945012  1.01792291]
[ 0.94768126 -2.01506523  1.12307292]
[ 0.98593321 -2.01224627  1.03240051]
[ 0.95445028 -1.95445635  1.01371649]


In [59]:
for model in s.non_parametric_bootstrap(X, y_logistic, n_sim=10):
    print(model.coef_)

[ 0.97689309 -1.97413283  1.10627714]
[ 0.95892487 -1.99413312  0.98665457]
[ 0.94129474 -1.94450942  1.00859525]
[ 0.93336539 -1.95264506  0.99312847]
[ 0.93940596 -1.91603988  0.99230598]
[ 0.95260326 -1.95432899  0.96347066]
[ 0.97404336 -1.95443378  1.07337741]
[ 0.93040456 -1.85128555  0.98612584]
[ 0.94250477 -1.9621798   1.02531186]
[ 0.9412844  -1.94538671  1.06160179]


## Poission Model

In [60]:
mu = np.exp(nu)
y_poisson = np.random.poisson(lam=mu, size=N)

In [61]:
model = GLM(family=Poisson())
model.fit(X, y_poisson)

In [62]:
model.coef_

array([ 0.9993738 , -2.01557039,  0.98708617])

In [63]:
model.coef_covariance_matrix_

array([[  5.19786341e-05,   5.41592437e-05,  -1.88130624e-05],
       [  5.41592437e-05,   1.00811556e-04,  -9.71528570e-07],
       [ -1.88130624e-05,  -9.71528570e-07,   6.18642911e-05]])

In [64]:
model.coef_standard_error_

array([ 0.00720962,  0.0100405 ,  0.00786539])

In [65]:
mod = statsmodels.discrete.discrete_model.Poisson(y_poisson, X)
res = mod.fit()
print(res.summary())

Optimization terminated successfully.
         Current function value: 1.857216
         Iterations 19
                          Poisson Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        Poisson   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Sun, 17 Sep 2017   Pseudo R-squ.:                  0.6578
Time:                        16:10:00   Log-Likelihood:                -18572.
converged:                       True   LL-Null:                       -54276.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9994      0.007    138.617      0.000       0.985       1.014
x1            -2.0156      0

In [66]:
s = Simulation(model)

In [67]:
s.sample(X, n_sim=10)

array([[  1.,   7.,   0., ...,   5.,   2.,   2.],
       [  3.,   8.,   1., ...,   5.,   0.,   5.],
       [  1.,   6.,   0., ...,   7.,   0.,   2.],
       ..., 
       [  0.,  10.,   1., ...,  10.,   1.,   6.],
       [  2.,   4.,   0., ...,   4.,   1.,   2.],
       [  3.,   5.,   0., ...,   3.,   1.,   3.]])

In [68]:
for model in s.parametric_bootstrap(X, n_sim=10):
    print(model.coef_)

[ 0.9962561  -2.02223879  0.98520933]
[ 1.00174874 -2.016179    0.98438619]
[ 1.01444728 -2.00367046  0.97880814]
[ 0.99162838 -2.01637286  1.00481727]
[ 1.00802374 -2.01063381  0.96973567]
[ 1.00579857 -2.01884458  0.9866297 ]
[ 0.9960399  -2.01891399  0.99412272]
[ 0.99357372 -2.0331467   0.99903938]
[ 0.99830196 -2.01604988  0.98445944]
[ 1.00074698 -2.02003424  0.98114143]


In [36]:
for model in s.non_parametric_bootstrap(X, y_poisson, n_sim=10):
    print(model.coef_)

[ 0.99944037 -1.95435021  0.97540803]
[ 1.03624533 -1.96567739  0.91936164]
[ 0.98336717 -1.96622823  1.00794745]
[ 1.02681072 -1.9644957   0.92060739]
[ 1.01231914 -1.94929026  0.96059324]
[ 1.01857949 -1.96132365  0.95859381]
[ 0.99512153 -1.97303933  0.99006156]
[ 1.01719772 -1.93460717  0.9529537 ]
[ 0.99912808 -1.95763946  0.97810085]
[ 1.00010555 -1.94437559  0.97117927]


## Poisson with Exposures

In [37]:
mu = np.exp(nu)
expos = np.random.uniform(0, 10, size=N)
y_poisson = np.random.poisson(lam=(mu*expos), size=N)

In [38]:
model = GLM(family=Poisson())
model.fit(X, y_poisson, offset=np.log(expos))

In [39]:
model.coef_

array([ 1.00325959, -1.99621054,  0.98932295])

In [40]:
model.coef_standard_error_

array([ 0.0084044 ,  0.01200987,  0.01120648])

## Gamma Regression

In [41]:
mu = np.exp(nu)
y_gamma = np.random.gamma(shape=2.0, scale=(mu / 2.0), size=N)

In [42]:
gamma_model = GLM(family=Gamma())
gamma_model.fit(X, y_gamma)

In [43]:
gamma_model.coef_

array([ 1.04538732, -1.99544524,  0.92854162])

In [44]:
gamma_model.coef_standard_error_

array([ 0.01960164,  0.02551379,  0.02568215])

In [45]:
gamma_model.dispersion_

0.54319477864783849

In [46]:
gamma_model = sm.GLM(y_gamma, X, 
                     family=sm.families.Gamma(
                         link=statsmodels.genmod.families.links.log))
res = gamma_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                   Gamma   Df Model:                            2
Link Function:                    log   Scale:                  0.499038505471
Method:                          IRLS   Log-Likelihood:                -13910.
Date:                Sun, 17 Sep 2017   Deviance:                       5430.3
Time:                        14:47:52   Pearson chi2:                 4.99e+03
No. Iterations:                     5                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0454      0.019     55.641      0.000       1.009       1.082
x1            -1.9954      0.024    -81.597      0.0

## Exponential Regression

In [47]:
mu = np.exp(nu)
y_exponential = np.random.exponential(scale=mu, size=N)

In [48]:
exponential_model = GLM(family=Gamma())
exponential_model.fit(X, y_exponential)

In [49]:
exponential_model.coef_

array([ 1.02547471, -2.03635258,  0.99990048])

In [50]:
exponential_model.coef_standard_error_

array([ 0.02824993,  0.03677053,  0.03701316])

In [51]:
exponential_model.dispersion_

1.1282497110609833

In [52]:
exponential_model = sm.GLM(y_exponential, X, 
                     family=sm.families.Gamma(
                         link=statsmodels.genmod.families.links.log))
res = exponential_model.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                   Gamma   Df Model:                            2
Link Function:                    log   Scale:                   0.98383654671
Method:                          IRLS   Log-Likelihood:                -15006.
Date:                Sun, 17 Sep 2017   Deviance:                       11279.
Time:                        14:47:52   Pearson chi2:                 9.84e+03
No. Iterations:                     6                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0255      0.026     38.873      0.000       0.974       1.077
x1            -2.0364      0.034    -59.305      0.0

## Linear Model with Correlated Predictors

In [53]:
N = 1000
X = np.empty(shape=(N, 3))
X[:, 0] = 1.0
X[:, 1] = np.random.uniform(size=N)
X[:, 2] = 0.5*X[:, 1] + np.random.uniform(-0.5, 0.5, size=N)
nu = 1 - 2*X[:, 1] + X[:, 2]

In [54]:
y = nu + np.random.normal(size=N)
model = GLM(family=Gaussian())
model.fit(X, y)

In [55]:
model.coef_

array([ 1.04961873, -2.09917034,  0.92403124])

In [56]:
model.coef_covariance_matrix_

array([[ 0.00390364, -0.00599316,  0.00019188],
       [-0.00599316,  0.01504632, -0.00598741],
       [ 0.00019188, -0.00598741,  0.01107235]])

In [57]:
model.coef_standard_error_

array([ 0.06247909,  0.12266344,  0.10522523])

In [58]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     146.9
Date:                Sun, 17 Sep 2017   Prob (F-statistic):           1.23e-56
Time:                        14:47:52   Log-Likelihood:                -1398.6
No. Observations:                1000   AIC:                             2803.
Df Residuals:                     997   BIC:                             2818.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0496      0.062     16.800      0.0